### **Filtragem por médias não-locais (vizinhança de Nagao-Matsuyama)**
###### Disciplinas: PDI Avançado / Processamento de Imagens SAR
###### Profª Drª Sidnei Sant'Anna/ Leila Fonseca  e José Claudio Mura
###### Discentes: Vinícius D'Lucas Bezerra e Queiroz , Lenon Silva de Oliveira.

A técnica de filtragem por médias não locais se baseia em procurar pixels ou patches na imagem que se assemelham à vizinhança de um pixel de referência e, fazendo uso de algum critério de similaridade, realiza a filtragem por meio do cálculo da média desses elementos. Segundo [1] a melhor forma de reduzir o efeito do speckle é realizar a média em objetos que sejam semelhantes na imagem. Dessa forma, os métodos de filtragem não local apresentam performances e resultados interessantes por levar em consideração que proximidades geométricas na imagem não implicam necessariamente em similaridades [1].  
Nagao e Matsuyama (1979)[2], por exemplo, propuseram um método de filtragem que tem a finalidade de reduzir o speckle sem borrar as bordas nem os detalhes imagem, sendo uma tentativa de resolver o conflito entre eliminação de ruído e degradação de bordas. Na prática, este método procura na vizinhança de um pixel de referência qual o patch que possui a menor variância e atribui na posição deste pixel o valor da média dos pixels desse mesmo patch.




Referências:


[1] Di Martino, G., Di Simone, A., Iodice, A., Riccio, D., & Ruello, G. Non-local means SAR despeckling based on scattering. 2015 IEEE International Geoscience and Remote Sensing Symposium (IGARSS) pp. 3172-3174, 2015. doi:10.1109/igarss.2015.7326491



[2] NAGAO, M.; MATSUYAMA, T. Edge preserving smoothing. Computer graphics and image processing, v. 9, n. 4, p. 394-407, 1979.





Instalando bibliotecas não-nativas

In [ ]:
!pip install rasterio

Importando outras bibliotecas

In [1]:
import rasterio as rio
import numpy as np

Definindo função de filtragem

In [2]:
def nlm_filter_ngmt(imagem): #Filtro de médias não-locais que utiliza a vizinhança de Nagao e Matsuyama (1979)

  # Verificando tamanho e largura da imagem e criando imagem de saída para filtragem
  # out é definido como uma imagem de zeros com quatro linhas e colunas a menos
  height, width = imagem.shape
  out = np.zeros((height + 4, width + 4), dtype=float)
  out[2: 2 + height, 2: 2 + width] = imagem.copy().astype(float)


  # Criação dos 8 patches de pixels da vizinhança de Nagao-Matsuyama
  qtd_iteracoes = 0
  for i in range(height):
      for j in range(width):
          patch1 = np.ravel(out[i + 1:i + 4, j:j + 2])
          patch1 = np.append(patch1, out[i + 2, j + 2])

          patch2 = np.ravel(out[i + 3:i + 5, j + 1:j + 4])
          patch2 = np.append(patch2, out[i + 2, j + 2])

          patch3 = np.ravel(out[i + 1:i + 4, j + 3:j + 5])
          patch3 = np.append(patch3, out[i + 2, j + 2])

          patch4 = np.ravel(out[i:i + 2, j + 1:j + 4])
          patch4 = np.append(patch4, out[i + 2, j + 2])

          patch5 = np.ravel(out[i:i + 2, j:j + 2])
          patch5 = np.append(patch5, [out[i + 2, j + 1],out[i + 1, j + 2],out[i + 2, j + 2]])

          patch6 = np.ravel(out[i + 3:i + 5, j:j + 2])
          patch6 = np.append(patch6, [out[i + 2, j + 1], out[i + 3, j + 2],out[i + 2, j + 2]])

          patch7 = np.ravel(out[i + 3:i + 5, j + 3:j + 5])
          patch7 = np.append(patch7, [out[i + 2, j + 3],out[i + 3, j + 2],out[i + 2, j + 2]])

          patch8 = np.ravel(out[i:i + 2, j + 3:j + 5])
          patch8 = np.append(patch8, [out[i + 1, j + 2], out[i + 2, j + 3],out[i + 2, j + 2]])

          #patch9 = np.ravel(out[i + 1: i + 4, j + 1:j + 4]) # Vizinhança de nagao-matsuyama modificada com patch central


          # Cálculo da variância como parâmetro de similaridade entre as 8 posições de vizinhança
          resultado_var = np.array([np.var(patch1), np.var(patch2), np.var(patch3), np.var(patch4), np.var(patch5), np.var(patch6), np.var(patch7), np.var(patch8)])
          patches = np.array([patch1,patch2,patch3,patch4,patch5,patch6,patch7,patch8])

          # Cálculo da média para a posição de menor variância
          out[2 + i, 2 + j] = np.mean(patches[resultado_var.argmin()])
          qtd_iteracoes += 1
  return out

**Executando função e exportando imagem como GeoTiFF**

In [3]:
# Mude este path, pelo caminho da imagem a ser filtrada
dataset = rio.open("G:/Meu Drive/UNIFESP/forest_eyes/4.imagens/imagens_processadas/data_sbsr/sentinel1_mosaic_30m_REC.TIF", mode='r')

In [4]:
name = 'sentinel_1'
out_path = "G:/Meu Drive/UNIFESP/forest_eyes/4.imagens/imagens_processadas/data_sbsr"
filtered_images = []
for i in dataset.indexes:
  print(f'>>> Iniciando filtragem da banda {i}...')
  out = nlm_filter_ngmt(dataset.read(i)) # Chamada da função dentro do loop.
  height, width = dataset.shape
  out = out[2:2 + height, 2:2 + width]# redimensionando as imagens para o mesmo tamanho.
  filtered_images.append(out)  # Armazenar banda filtrada
  print(f'>>> Banda {i} filtrada!! \n')

# Converter lista de bandas filtradas para um array numpy com dimensões (bandas, altura, largura)
stacked_image = np.stack(filtered_images)

# Exportar o stack de bandas filtradas
tiff_name = f'{out_path}/{name}_stack_ngmt.TIF'

# Abrir um novo arquivo GeoTIFF com o número correto de bandas
with rio.open(tiff_name,
              'w',
              driver='GTiff',
              height=stacked_image.shape[1],
              width=stacked_image.shape[2],
              count=len(dataset.indexes),  # número de bandas
              dtype=stacked_image.dtype,
              crs=dataset.crs,
              nodata=None,  # Ajuste se necessário
              transform=dataset.transform) as dst:

    # Escrever cada banda no arquivo
    for i in range(stacked_image.shape[0]):
        dst.write(stacked_image[i], i + 1)
print(">>> Fim da filtragem!")

>>> Iniciando filtragem banda 1...
>>> Banda 1 filtrada!...

>>> Iniciando filtragem banda 2...
>>> Banda 2 filtrada!...

